In [1]:
import GPy
import numpy as np
from desdeo_problem.Problem import DataProblem

from desdeo_problem.surrogatemodels.SurrogateModels import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from desdeo_emo.EAs.OfflineRVEA import ProbRVEAv3

from desdeo_problem.testproblems.TestProblems import test_problem_builder
from pyDOE import lhs
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pygmo import non_dominated_front_2d as nd2

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import time

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


In [2]:
problem_name = "DTLZ2"
nvars = 10
nobjs = 2
nsamples = 100
prob = test_problem_builder(problem_name, nvars, nobjs)

x = lhs(nvars, nsamples)
x_test = lhs(nvars, int(nsamples*0.2))

y = prob.evaluate(x)
y=np.asarray(y[0])

y_test = prob.evaluate(x_test)
y_test=np.asarray(y_test[0])

x_names = [f'x{i}' for i in range(1,nvars)]
y_names = ["f1", "f2"]

In [4]:
print("Building full GP...")
start = time.time()
m_full = GPy.models.GPRegression(x,y)
m_full.optimize('bfgs')
end = time.time()
print("Time taken:")
print(end - start)
#m_full.plot()
#print(m_full)
y_prediction = np.asarray(m_full.predict(x_test)[0])
#y_error = np.subtract(y_prediction, y)
print("R squared")
print(r2_score(y_test[:,0], y_prediction[:,0]))
print(r2_score(y_test[:,1], y_prediction[:,1]))
print("RMSE")
print(mean_squared_error(y_test[:,0], y_prediction[:,0]))
print(mean_squared_error(y_test[:,1], y_prediction[:,1]))

2020-09-22 13:09:08,598 - GP - INFO - initializing Y
2020-09-22 13:09:08,606 - GP - INFO - initializing inference method
2020-09-22 13:09:08,608 - GP - INFO - adding kernel and likelihood as parameters
Building full GP...
Time taken:
1.460864543914795
R squared
0.946348613992138
0.9346854338928551
RMSE
0.018489763642121772
0.019297062928611455


In [5]:
m_full.predict(x_test)


(array([[1.2055012 , 0.7158557 ],
        [1.7681938 , 0.00642852],
        [1.80762319, 0.86840045],
        [1.35051563, 1.08470514],
        [0.19697805, 1.7047139 ],
        [0.23793282, 1.58746368],
        [1.57836414, 0.7651921 ],
        [1.8001763 , 0.19031812],
        [1.18856523, 1.26525903],
        [0.81310956, 1.6295713 ],
        [0.89602573, 1.50176056],
        [1.85935922, 0.57323326],
        [0.95606768, 1.08267087],
        [0.51972041, 1.89509629],
        [0.85132155, 1.41716339],
        [1.58830614, 0.50075037],
        [1.54499251, 0.86313501],
        [1.5164268 , 1.41523077],
        [0.13854622, 1.66565692],
        [0.6864802 , 1.81056954]]),
 array([[0.00997285],
        [0.01216785],
        [0.03747719],
        [0.02227567],
        [0.01571174],
        [0.01014839],
        [0.01348219],
        [0.02277761],
        [0.01451101],
        [0.01605539],
        [0.01273579],
        [0.01736584],
        [0.01006444],
        [0.01679543],
        [0

In [3]:
# Sparse GP models
Z = np.random.rand(500,nvars)
m = GPy.models.SparseGPRegression(x,y,Z=Z)
start = time.time()
m.inducing_inputs.fix()
m.optimize('bfgs')
end = time.time()
print("Time taken:")
print(end - start)
y_prediction = np.asarray(m.predict(x_test)[0])
print("R squared")
print(r2_score(y_test[:,0], y_prediction[:,0]))
print(r2_score(y_test[:,1], y_prediction[:,1]))
print("RMSE")
print(mean_squared_error(y_test[:,0], y_prediction[:,0]))
print(mean_squared_error(y_test[:,1], y_prediction[:,1]))



2020-08-14 01:13:15,824 - GP - INFO - initializing Y
2020-08-14 01:13:15,826 - GP - INFO - initializing inference method
2020-08-14 01:13:15,830 - GP - INFO - adding kernel and likelihood as parameters
2020-08-14 01:13:15,835 - sparse gp - INFO - Adding Z as parameter
Time taken:
5.2162556648254395
R squared
0.9579992308686689
0.9627690579852343
RMSE
0.012058411549233502
0.013633993249994706


In [6]:
start = time.time()
m.randomize()
m.Z.unconstrain()
m.optimize('bfgs')
end = time.time()
print("Time taken:")
print(end - start)
y_prediction = np.asarray(m.predict(x_test)[0])
print("R squared")
print(r2_score(y_test[:,0], y_prediction[:,0]))
print(r2_score(y_test[:,1], y_prediction[:,1]))
print("RMSE")
print(mean_squared_error(y_test[:,0], y_prediction[:,0]))
print(mean_squared_error(y_test[:,1], y_prediction[:,1]))

Time taken:
169.37006163597107
R squared
0.9575921439554733
0.9530028360321182
RMSE
0.012526382724409405
0.015486414758992567


In [12]:
print(m.Z)

   |        
  [474   7]  |                 0.50394615  |     fixed     |        
  [474   8]  |                 0.73309701  |     fixed     |        
  [474   9]  |                 0.06656304  |     fixed     |        
  [475   0]  |                 0.75340574  |     fixed     |        
  [475   1]  |                 0.62512730  |     fixed     |        
  [475   2]  |                 0.37252297  |     fixed     |        
  [475   3]  |                 0.19813449  |     fixed     |        
  [475   4]  |                 0.91589623  |     fixed     |        
  [475   5]  |                 0.32252346  |     fixed     |        
  [475   6]  |                 0.56606702  |     fixed     |        
  [475   7]  |                 0.06186568  |     fixed     |        
  [475   8]  |                 0.48615496  |     fixed     |        
  [475   9]  |                 0.60200377  |     fixed     |        
  [476   0]  |                 0.09485961  |     fixed     |        
  [476   1]  |       

In [41]:
print(m.log_likelihood(), m_full.log_likelihood())
print(m)

[[1598.70615983]] 4095.845078250409

Name : sparse_gp
Objective : -1598.7061598329747
Number of Parameters : 503
Number of Optimization Parameters : 503
Updates : True
Parameters:
  sparse_gp.               |                 value  |  constraints  |  priors
  inducing_inputs          |              (50, 10)  |               |        
  rbf.variance             |    26.490103449601904  |      +ve      |        
  rbf.lengthscale          |     5.456090552872434  |      +ve      |        
  Gaussian_noise.variance  |  0.006859055478648036  |      +ve      |        


In [44]:
print(m.log_likelihood(), m_full.log_likelihood())
print(m)

[[2325.95403679]] 4095.845078250409

Name : sparse_gp
Objective : -2325.9540367942536
Number of Parameters : 1003
Number of Optimization Parameters : 1003
Updates : True
Parameters:
  sparse_gp.               |                  value  |  constraints  |  priors
  inducing_inputs          |              (100, 10)  |               |        
  rbf.variance             |      29.39059191286705  |      +ve      |        
  rbf.lengthscale          |     3.6371820755805655  |      +ve      |        
  Gaussian_noise.variance  |  0.0022828089308227577  |      +ve      |        


In [47]:
print(m.log_likelihood(), m_full.log_likelihood())
print(m)

[[258.68560463]] 4095.845078250409

Name : sparse_gp
Objective : -258.68560462854657
Number of Parameters : 103
Number of Optimization Parameters : 103
Updates : True
Parameters:
  sparse_gp.               |               value  |  constraints  |  priors
  inducing_inputs          |            (10, 10)  |               |        
  rbf.variance             |  31.098932087574315  |      +ve      |        
  rbf.lengthscale          |  28.188888205484627  |      +ve      |        
  Gaussian_noise.variance  |  0.0398449307299818  |      +ve      |        


In [50]:
print(m.log_likelihood(), m_full.log_likelihood())
print(m)

[[3848.26264372]] 4095.845078250409

Name : sparse_gp
Objective : -3848.26264372468
Number of Parameters : 5003
Number of Optimization Parameters : 5003
Updates : True
Parameters:
  sparse_gp.               |                   value  |  constraints  |  priors
  inducing_inputs          |               (500, 10)  |               |        
  rbf.variance             |       40.67628849875355  |      +ve      |        
  rbf.lengthscale          |      2.9647226533433755  |      +ve      |        
  Gaussian_noise.variance  |  0.00013710403503997394  |      +ve      |        


In [28]:
vals = np.array([[0, 0],
                 [1, 0],
                 [2, 0],
                 [0, 1],
                 [1, 1],
                 [2, 1],
                 [0, 2],
                 [1, 2],
                 [2, 2],
                 [0, 3],
                 [1, 3],
                 [2, 3]])
vals3 = np.array([[0, 0],
                 [1, 0],
                 [2, 0],
                 [0, 1],
                 [1, 1],
                 [2, 1],
                 [0, 2],
                 [1, 2],
                 [2, 2],
                 [0, 3],
                 [1, 3],
                 [2, 3]])
vals2 = [[0,1],[2,3]]
valx = None
for i in range(np.shape(vals2)[0]):
    pos=np.where((vals == vals2[i]).all(axis=1))
    vind=vals3[pos]
    if valx is None:
        valx=vind
    else:
        valx=np.vstack((valx,vind))
print(valx)


[[0 1]]
[[2 3]]
[[0 1]
 [2 3]]
